1 Sentiment and Thematic Analysis

Introduction — Sentiment Analysis
In this phase of the project, we aim to quantify customer opinions from Google Play reviews of Ethiopian bank apps. After cleaning and normalizing the review text, we apply sentiment analysis to classify feedback as positive, negative, or neutral. This process provides a measurable understanding of user satisfaction and highlights areas for improvement. By combining sentiment scores with review metadata, we can identify trends across banks, app versions, and time, laying the groundwork for deeper thematic and semantic insights.


In [10]:
import sys, os
project_root = os.path.abspath("..")  # adjust if needed
if project_root not in sys.path:
    sys.path.append(project_root)

# Import the SentimentAnalyzer
from src.sentiment import SentimentAnalyzer

In [11]:
analyzer = SentimentAnalyzer()

In [12]:
# Add sentiment to cleaned reviews and save results
df_sentiment = analyzer.add_sentiment()

Sentiment analysis completed. Saved to data/processed/sentiment.csv


In [13]:
agg_sentiment = analyzer.aggregate_by_bank()
agg_sentiment

,bank,total_reviews,positive_reviews,negative_reviews,neutral_reviews,mean_polarity
0,Bank of Abyssinia,286,121,63,102,0.102740
1,Commercial Bank of Ethiopia,296,146,27,123,0.229599
2,Dashen Bank,286,171,44,71,0.230363


2 Thematic analysis

In [20]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from collections import Counter

# Load spaCy
nlp = spacy.load("en_core_web_sm")

# Load dataset
df = pd.read_csv("data/processed/cleaned.csv")

# --- FIX 1: Ensure content exists ---
df = df.dropna(subset=["content"])

# --- FIX 2: Auto-create review_id if missing ---
if "review_id" not in df.columns:
    df["review_id"] = df.index.astype(str)

# --- TEXT PREPROCESSING ---
def preprocess_text(text):
    doc = nlp(text)
    tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and token.is_alpha
    ]
    return " ".join(tokens)

df["processed_text"] = df["content"].apply(preprocess_text)

# --- SENTIMENT ANALYSIS ---
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.05:
        label = "positive"
    elif polarity < -0.05:
        label = "negative"
    else:
        label = "neutral"
    return polarity, label

df["sentiment_score"], df["sentiment_label"] = zip(*df["content"].apply(get_sentiment))

# --- THEME KEYWORDS ---
theme_keywords = {
    "Account Access Issues": ["login", "otp", "password", "authentication", "access"],
    "Transaction Performance": ["transfer", "payment", "failed", "slow", "transaction"],
    "User Interface & Experience": ["ui", "interface", "navigation", "layout", "experience"],
    "Customer Support": ["support", "help", "service", "complaint", "response"],
    "Feature Requests": ["feature", "request", "option", "add", "missing"]
}

def assign_themes(text):
    words = text.split()
    themes = [theme for theme, keywords in theme_keywords.items() if any(k in words for k in keywords)]
    return ", ".join(themes) if themes else "Uncategorized"

df["identified_theme(s)"] = df["processed_text"].apply(assign_themes)

# --- FIX 3: Select only the required final columns ---
final_df = df[[
    "review_id",
    "content",
    "sentiment_label",
    "sentiment_score",
    "identified_theme(s)"
]].rename(columns={"content": "review_text"})

# SAVE OUTPUT
final_df.to_csv("data/processed/sentiment_and_themes.csv", index=False)

print("✅ FINAL CSV SAVED SUCCESSFULLY!")
print(final_df.head())

✅ FINAL CSV SAVED SUCCESSFULLY!
  review_id          review_text sentiment_label  sentiment_score  \
0         0        very good app        positive         0.910000   
1         1            excellent        positive         1.000000   
2         2                 good        positive         0.700000   
3         3            CBE ይለያል።         neutral         0.000000   
4         4  it's special for me        positive         0.357143   

  identified_theme(s)  
0       Uncategorized  
1       Uncategorized  
2       Uncategorized  
3       Uncategorized  
4       Uncategorized  


Keyword Extraction with spaCy

In [21]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

def extract_spacy_keywords(text):
    doc = nlp(text)
    keywords = [
        token.lemma_.lower()
        for token in doc
        if token.pos_ in ["NOUN", "VERB", "ADJ"] 
        and not token.is_stop 
        and token.is_alpha
    ]
    return keywords

df["spacy_keywords"] = df["processed_text"].apply(extract_spacy_keywords)

# Count all keywords
all_keywords = [kw for sublist in df["spacy_keywords"] for kw in sublist]
keyword_freq_spacy = Counter(all_keywords).most_common(30)

print("\n🔥 Top spaCy Keywords:")
print(keyword_freq_spacy)



🔥 Top spaCy Keywords:
[('app', 315), ('good', 136), ('work', 107), ('bank', 88), ('banking', 70), ('use', 61), ('update', 61), ('time', 57), ('bad', 54), ('mobile', 41), ('money', 38), ('transfer', 37), ('service', 35), ('fix', 34), ('need', 33), ('easy', 32), ('user', 31), ('application', 31), ('transaction', 31), ('option', 30), ('open', 28), ('account', 27), ('experience', 27), ('great', 26), ('slow', 26), ('problem', 25), ('phone', 25), ('fast', 25), ('feature', 23), ('issue', 23)]


Keyword Extraction Using TF-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# TF-IDF for 1-gram + 2-gram
vectorizer = TfidfVectorizer(
    max_features=50, 
    ngram_range=(1, 2)
)

tfidf_matrix = vectorizer.fit_transform(df["processed_text"])
feature_names = vectorizer.get_feature_names_out()

# Get mean TF-IDF for each keyword
mean_scores = tfidf_matrix.mean(axis=0).A1
tfidf_scores = list(zip(feature_names, mean_scores))

# Sort by score
tfidf_keywords = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:30]

print("\n🔥 Top TF-IDF Keywords & Bigrams:")
print(tfidf_keywords)



🔥 Top TF-IDF Keywords & Bigrams:
[('app', np.float64(0.16443354834882198)), ('good', np.float64(0.09156474873761535)), ('bank', np.float64(0.06042833274551439)), ('work', np.float64(0.057746208600472596)), ('update', np.float64(0.03580773147282099)), ('bad', np.float64(0.031573452742465924)), ('use', np.float64(0.031127311352324563)), ('like', np.float64(0.03026478393333195)), ('banking', np.float64(0.029346101570805146)), ('time', np.float64(0.027830029584740134)), ('good app', np.float64(0.025329329237439715)), ('fast', np.float64(0.024945327269310896)), ('easy', np.float64(0.023147659371443766)), ('need', np.float64(0.02307421683177005)), ('application', np.float64(0.021861480329199608)), ('mobile', np.float64(0.021677127559114478)), ('nice', np.float64(0.02154868769316524)), ('amazing', np.float64(0.02130440679272414)), ('service', np.float64(0.0209799935238966)), ('transaction', np.float64(0.020376884862434455)), ('dashen', np.float64(0.019982813771605754)), ('cbe', np.float64(0.

1 Define Themes + Keywords (Theme Dictionary)

In [24]:
theme_dict = {
    "Account Access Issues": [
        "login", "otp", "password", "authentication", "access", "session"
    ],
    "Transaction Performance": [
        "transfer", "payment", "failed", "slow", "delay", "transaction", "balance"
    ],
    "User Interface & Experience": [
        "ui", "interface", "navigation", "layout", "design", "screen", "button"
    ],
    "Customer Support": [
        "support", "response", "help", "service", "complaint"
    ],
    "Feature Requests": [
        "feature", "request", "option", "add", "missing", "update"
    ]
}

2. Assign Themes to Each Review (Based on spaCy Keywords)

In [25]:
def assign_themes(keyword_list):
    review_themes = []

    for theme, keywords in theme_dict.items():
        if any(kw in keyword_list for kw in keywords):
            review_themes.append(theme)

    if not review_themes:
        return ["Other"]

    return review_themes

df["themes"] = df["spacy_keywords"].apply(assign_themes)

3. Generate Theme Counts Per Bank (Core Requirement)

In [26]:
from collections import Counter
import pandas as pd

bank_theme_summary = []

for bank, group in df.groupby("bank"):
    all_themes = [t for sub in group["themes"] for t in sub]
    theme_count = Counter(all_themes)

    for theme, count in theme_count.items():
        bank_theme_summary.append({
            "bank": bank,
            "theme": theme,
            "count": count
        })

bank_theme_df = pd.DataFrame(bank_theme_summary)
bank_theme_df = bank_theme_df.sort_values(["bank","count"], ascending=[True, False])


Pick Top 3–5 Themes per Bank (Final Clustering Requirement)

In [31]:
top_themes_per_bank = (
    bank_theme_df.groupby("bank")
    .head(5)
    .reset_index(drop=True)
)

In [35]:
final_df = df[[
    "review_id",
    "content",
    "sentiment_label",
    "sentiment_score",
    "bank",
    "themes",
    "spacy_keywords",
    "processed_text"
]]

final_df.to_csv("data/processed/final_reviews_with_themes.csv", index=False)
print("Saved: final_reviews_with_themes.csv")

Saved: final_reviews_with_themes.csv


In [38]:
print(top_themes_per_bank)

                           bank                        theme  count
0             Bank of Abyssinia                        Other    218
1             Bank of Abyssinia      Transaction Performance     30
2             Bank of Abyssinia             Feature Requests     30
3             Bank of Abyssinia             Customer Support     14
4             Bank of Abyssinia        Account Access Issues     11
5   Commercial Bank of Ethiopia                        Other    229
6   Commercial Bank of Ethiopia             Feature Requests     34
7   Commercial Bank of Ethiopia      Transaction Performance     24
8   Commercial Bank of Ethiopia             Customer Support     13
9   Commercial Bank of Ethiopia        Account Access Issues      9
10                  Dashen Bank                        Other    203
11                  Dashen Bank      Transaction Performance     35
12                  Dashen Bank             Feature Requests     32
13                  Dashen Bank  User Interface 

Insights and Recommendations

🧠 1. Define Driver / Pain Point Rules

In [39]:
def classify_insight(row):
    if row["sentiment_label"] == "positive":
        return "driver"
    elif row["sentiment_label"] == "negative":
        return "pain_point"
    return None

In [40]:
df["insight_type"] = df.apply(classify_insight, axis=1)


🧠 2. Extract Top Drivers & Pain Points per Bank

In [41]:
from collections import Counter

insight_summary = []

for bank, group in df.groupby("bank"):

    # DRIVER THEMES (positive reviews)
    pos_themes = [t for sub in group[group["insight_type"]=="driver"]["themes"] for t in sub]
    pos_count = Counter(pos_themes).most_common(3)

    # PAIN POINT THEMES (negative reviews)
    neg_themes = [t for sub in group[group["insight_type"]=="pain_point"]["themes"] for t in sub]
    neg_count = Counter(neg_themes).most_common(3)

    insight_summary.append({
        "bank": bank,
        "drivers": pos_count,
        "pain_points": neg_count
    })

insights_df = pd.DataFrame(insight_summary)


In [42]:
print(insights_df)

                          bank  \
0            Bank of Abyssinia   
1  Commercial Bank of Ethiopia   
2                  Dashen Bank   

                                             drivers  \
0  [(Other, 83), (Feature Requests, 13), (Transac...   
1  [(Other, 108), (Feature Requests, 18), (Transa...   
2  [(Other, 117), (User Interface & Experience, 1...   

                                         pain_points  
0  [(Other, 40), (Transaction Performance, 13), (...  
1  [(Other, 11), (Transaction Performance, 7), (F...  
2  [(Other, 22), (Transaction Performance, 14), (...  


In [43]:
def format_insights(row):
    driver_list = ", ".join([f"{t} ({c})" for t,c in row["drivers"]])
    pain_list = ", ".join([f"{t} ({c})" for t,c in row["pain_points"]])

    return f"""
    📌 **{row['bank']} Insights**
    ▶ **Drivers:** {driver_list}
    ▶ **Pain Points:** {pain_list}
    """

for _, r in insights_df.iterrows():
    print(format_insights(r))
    print("\n" + "-"*80 + "\n")



    📌 **Bank of Abyssinia Insights**
    ▶ **Drivers:** Other (83), Feature Requests (13), Transaction Performance (11)
    ▶ **Pain Points:** Other (40), Transaction Performance (13), Feature Requests (10)
    

--------------------------------------------------------------------------------


    📌 **Commercial Bank of Ethiopia Insights**
    ▶ **Drivers:** Other (108), Feature Requests (18), Transaction Performance (10)
    ▶ **Pain Points:** Other (11), Transaction Performance (7), Feature Requests (4)
    

--------------------------------------------------------------------------------


    📌 **Dashen Bank Insights**
    ▶ **Drivers:** Other (117), User Interface & Experience (19), Feature Requests (18)
    ▶ **Pain Points:** Other (22), Transaction Performance (14), Feature Requests (6)
    

--------------------------------------------------------------------------------

